In [3]:
import sys

from os import getcwd
from os.path import dirname, abspath


sys.path.append(dirname((abspath(getcwd()))))

In [29]:
import numpy as np
import pandas as pd

from joblib import load

from school_budget.config import LABELS
from school_budget.data import multilabel_sample_dataframe, multilabel_train_test_split
from school_budget.models import predict, top_n_predictions_ids

In [20]:
df = pd.read_csv("../datasets/TrainingData.csv", index_col=0)
NON_LABELS = [c for c in df.columns if c not in LABELS]
SAMPLE_SIZE = 40000
sampling = multilabel_sample_dataframe(
    df,
    pd.get_dummies(df[LABELS]),
    size=SAMPLE_SIZE,
    min_count=25,
    seed=43
)
dummy_labels = pd.get_dummies(sampling[LABELS])
X_train, X_test, y_train, y_test = multilabel_train_test_split(
    sampling[NON_LABELS],
    dummy_labels,
    0.2,
    min_count=3,
    seed=43
)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400277 entries, 134338 to 415831
Data columns (total 25 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Function                400277 non-null  object 
 1   Use                     400277 non-null  object 
 2   Sharing                 400277 non-null  object 
 3   Reporting               400277 non-null  object 
 4   Student_Type            400277 non-null  object 
 5   Position_Type           400277 non-null  object 
 6   Object_Type             400277 non-null  object 
 7   Pre_K                   400277 non-null  object 
 8   Operating_Status        400277 non-null  object 
 9   Object_Description      375493 non-null  object 
 10  Text_2                  88217 non-null   object 
 11  SubFund_Description     306855 non-null  object 
 12  Job_Title_Description   292743 non-null  object 
 13  Text_3                  109152 non-null  object 
 14  Text_4         

In [30]:
model = load("model-1.1.0.joblib")
predictions = predict(model, X_test)
# Format correctly in new DataFrame: prediction_df
prediction_df = pd.DataFrame(
    columns=pd.get_dummies(df[LABELS]).columns,
    index=X_test.index,
    data=predictions
)

In [76]:
top_n_pred = top_n_predictions_ids(predictions)
print(top_n_pred.shape)
class_labels = model.classes_
print(prediction_df.columns[top_n_pred])
predicted_sorted = predictions[np.arange(top_n_pred.shape[0]),top_n_pred.T].T
print(predicted_sorted[-3:])

for i,d in enumerate(X_test):
    for p in top_n_pred[i]:
        print(prediction_df.columns[model.classes_[p]],"(",f"{predictions[i][p]:.0%}",")")

(8000, 3)
[['Position_Type_Substitute' 'Operating_Status_PreK-12 Operating'
  'Object_Type_Benefits']
 ['Position_Type_Substitute' 'Operating_Status_PreK-12 Operating'
  'Pre_K_NO_LABEL']
 ['Object_Type_Benefits' 'Operating_Status_PreK-12 Operating'
  'Pre_K_NO_LABEL']
 ...
 ['Object_Type_Substitute Compensation'
  'Operating_Status_PreK-12 Operating' 'Use_Instruction']
 ['Object_Type_Base Salary/Compensation' 'Function_Security & Safety'
  'Pre_K_NO_LABEL']
 ['Pre_K_NO_LABEL' 'Object_Type_Contracted Services'
  'Operating_Status_PreK-12 Operating']]
[[0.99937687 0.99951954 0.99995407]
 [0.98149012 0.99539182 0.99993606]
 [0.94287294 0.98514529 0.99103023]]
Position_Type_Substitute ( 100% )
Operating_Status_PreK-12 Operating ( 100% )
Object_Type_Benefits ( 100% )
Position_Type_Substitute ( 100% )
Operating_Status_PreK-12 Operating ( 100% )
Pre_K_NO_LABEL ( 100% )
Object_Type_Benefits ( 99% )
Operating_Status_PreK-12 Operating ( 99% )
Pre_K_NO_LABEL ( 100% )
Reporting_Non-School ( 100% 

/var/folders/zh/63z3wwyj50gfb13kmq48bdvc0000gn/T/ipykernel_3879/214869044.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  print(prediction_df.columns[top_n_pred])


In [25]:
f_row = X_test.iloc[0].to_frame()
print(X_test.iloc[0].to_frame())
predictions = model.predict_proba(X_test.head(1))
print(type(X_test.head(1)))
print(X_test.head(1).columns)
print(X_test.iloc[0].to_frame().T.columns)
print(predictions)
print(predictions.shape)
# pl.predict_proba(f_row)

                                                 662
Object_Description                 EMPLOYEE BENEFITS
Text_2                               EARLY EDUCATION
SubFund_Description          FEDERAL GDPG FUND - FY 
Job_Title_Description   Teacher,Retrd Shrt Term Sub 
Text_3                                       Regular
Text_4                                           NaN
Sub_Object_Description                           NaN
Location_Description                             NaN
FTE                                              NaN
Function_Description                      HEAD START
Facility_or_Department                           NaN
Position_Extra            PROFESSIONAL-INSTRUCTIONAL
Total                                           0.63
Program_Description                  EARLY CHILDHOOD
Fund_Description                                 NaN
Text_1                           REGULAR INSTRUCTION
<class 'pandas.core.frame.DataFrame'>
Index(['Object_Description', 'Text_2', 'SubFund_Description',